In [23]:
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from typing import List, Optional



In [26]:
import toml
with open('secrets.toml','r') as s:
    secrets = toml.load(s)


In [30]:
email = secrets.get('email')
password = secrets.get('epassword')

In [31]:
from typing import Optional, List
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

def send_compliance_email(
    sender_email: str,
    sender_password: str,
    recipient_email: List[str],  # <-- accept list of recipients
    compliance_status: str,
    report: str,
    smtp_server: str = "smtp.gmail.com",
    smtp_port: int = 587,
    use_html: bool = False,
    attachments: Optional[List[str]] = None
) -> bool:
    try:
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = ", ".join(recipient_email)  # comma-separated string for headers
        msg['Subject'] = "Compliance Status Report"

        if use_html:
            body = f"""
            <html>
            <body>
                <h2>Compliance Status: {compliance_status}</h2>
                <p><strong>Report:</strong></p>
                <pre>{report}</pre>
            </body>
            </html>
            """
            msg.attach(MIMEText(body, 'html'))
        else:
            body = f"Compliance Status: {compliance_status}\n\nReport:\n{report}"
            msg.attach(MIMEText(body, 'plain'))

        if attachments:
            for filepath in attachments:
                if os.path.isfile(filepath):
                    with open(filepath, 'rb') as file:
                        part = MIMEApplication(file.read(), Name=os.path.basename(filepath))
                        part['Content-Disposition'] = f'attachment; filename="{os.path.basename(filepath)}"'
                        msg.attach(part)
                else:
                    print(f"Attachment not found: {filepath}")

        with smtplib.SMTP(smtp_server, smtp_port, timeout=10) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())  # pass list here

        print("Email sent successfully.")
        return True

    except smtplib.SMTPException as e:
        print(f"SMTP error occurred: {e}")
        return False
    except Exception as ex:
        print(f"An unexpected error occurred: {ex}")
        return False


In [32]:
send_compliance_email(
    sender_email=email,
    sender_password=password,
    recipient_email= [ 
        'drejones021@gmail.com'
                        ],
    compliance_status="AAHHHH",
    report="this is a test of the notification system",
    use_html=True
    #attachments=["/path/to/report.pdf", "/path/to/logfile.txt"]
)

Email sent successfully.


True